In [1]:
!pip install pylatex

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import rankdata
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.libqsturng import psturng
from itertools import combinations
import sqlite3
from ast import literal_eval
from pathlib import Path
from pylatex import Document, NoEscape
from IPython.display import display, Latex
from ana_utils import *

METRIC = "r2_score" 
output_dir = Path("./cache/analysis/")
output_dir.mkdir(parents=True, exist_ok=True)

In [3]:
def df_to_booktabs(df: pd.DataFrame, renaming_conventions: dict, title: str, optimization: str=None, caption: str = "", label: str = "") -> str:
    """
    Convert a Pandas DataFrame to a LaTeX table using the booktabs style.
    """
    df = df[sorted(df.columns, reverse=True)]
    float_cols = df.select_dtypes(include=['float', 'float64', 'float32']).columns
    int_cols = df.select_dtypes(include=['int', 'int64', 'int32']).columns
    if optimization is not None:
        optimum = np.min if optimization == "min" else np.max
        df[float_cols] = df[float_cols].apply(
            lambda col: col.map(
                lambda x: f"\\textbf{{{x:.2f}}}" if x == optimum(col) else f"{x:.2f}"
            )
        )
        df[int_cols] = df[int_cols].apply(
            lambda col: col.map(
                lambda x: f"\\textbf{{{x:.0f}}}" if x == optimum(col) else f"{x:.0f}"
            )
        )

    latex_str = "\\begin{table}[t!]\n"
    latex_str += "\\centering\n"

    # Convert DataFrame to LaTeX with booktabs
    latex_str += df.to_latex(index=False, escape=False, 
                             header=True, 
                             column_format="l" + "r" * (len(df.columns) - 1),
                             longtable=False, 
                             bold_rows=False,
                             multicolumn=True,
                             multicolumn_format='c',
                             float_format="%.2f",
                             )
                             
    if caption:
        latex_str += f"\\caption{{{caption}}}\n"
    if label:
        latex_str += f"\\label{{{label}}}\n"
    latex_str += "\\end{table}"


    with open(output_dir/("tab_"+title+".tex"), "w") as file:
        file.write(latex_str)


    return latex_str

def render_latex_table(latex_code, output_file="output"):
    # Create a LaTeX document
    doc = Document()
    doc.preamble.append(NoEscape(r"\usepackage{booktabs}"))
    doc.append(NoEscape(r"\section*{Example Table}"))
    doc.append(NoEscape(latex_code))  # Append the LaTeX table as raw string
    # Compile to PDF
    doc.generate_pdf(output_dir / output_file, clean_tex=True)
    print(f"PDF generated: {output_file}")

In [5]:
# Average Ranking of each method
# Load results from db
conn_ctr = sqlite3.connect(ctr_path)
conn_vimet = sqlite3.connect(vimet_path)
df = pd.read_sql_query(f"SELECT data_config_hash, data_id, search_space_name as search_space, model, {METRIC}  FROM results", conn_ctr)
# Calculate mean R2 score
df[METRIC] = df[METRIC].apply(literal_eval)
df[METRIC] = df[METRIC].apply(np.mean)

# all unique values
print(len(df))
ds_hashs = df["data_config_hash"].unique()
ds_ids = df["data_id"].unique()
model_names = df["search_space"].unique()
df = df.drop_duplicates(
    subset=["data_config_hash", "data_id", "search_space"],
    keep="last"
)

# full cartesian product
full_index = pd.MultiIndex.from_product(
    [ds_hashs, ds_ids, model_names],
    names=["data_config_hash", "data_id", "search_space"]
)

# set index to those columns
df = df.set_index(["data_config_hash", "data_id", "search_space"])

# reindex and fill missing
df = df.reindex(full_index, fill_value=-np.inf).reset_index()
df["failure"] = df[METRIC] == -np.inf
print(len(df))

# Group Data by data_id data_config_hash
df['rank'] = df.groupby(['data_id', 'data_config_hash'])[METRIC].rank(ascending=False)
df['clean_rank'] = df[df[METRIC] != -np.inf].groupby(['data_id', 'data_config_hash'])[METRIC].rank(ascending=False)

# Calculate average rank for each method
df = (
    df.groupby("search_space", as_index=False)
      .agg(mean_rank=("rank", "mean"),
           total_failures=("failure", "sum"),
           mean_clean_rank=("clean_rank", "mean")
           )
)

df[["Model", "Avg Rank", "#Failures"]] = df[["search_space", "mean_rank", "total_failures"]]
df = df[["Model", "Avg Rank", "#Failures"]]
df["#Failures"] = df["#Failures"].astype(int)

table = df_to_booktabs(df, renaming_conventions={}, title="Ranking", optimization="min", caption="Average ranking of different models across several benchmarks.", label="tab:Ranking")
try:
    render_latex_table("\n\n".join([table]))
except Exception as e:
    print(f"Error generating LaTeX table: {e}")
# TODO Join with custom Benchmark tables
df

1075
1155
Error generating LaTeX table: No LaTex compiler was found
Either specify a LaTex compiler or make sure you have latexmk or pdfLaTex installed.


,Model,Avg Rank,#Failures
0,AdaBoost,13.257143,0
1,AutoGluon(10min),4.914286,0
2,AutoGluon(1h),3.085714,0
3,AutoGluon(1min),9.028571,0
4,AutoGluon(2h),25.028571,26
5,AutoGluon(30min),3.971429,0
6,AutoGluon(5min),5.542857,1
7,Bagging,19.571429,0
8,BayesianRidge,17.371429,0
9,CatBoost,9.114286,0


In [8]:
df = pd.read_sql_query(f"SELECT data_config_hash, data_id, model, {METRIC}  FROM results", conn_ctr)
df[df["model"]=="AutoGluon"]

,data_config_hash,data_id,model,r2_score
617,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,24,AutoGluon,"[0.09995257592789297, 0.5998551676905699, 0.67..."
666,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,6,AutoGluon,"[0.3911891524882337, 0.34054836929598264, 0.45..."
668,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,9,AutoGluon,"[0.53933184775492, 0.6402262841919886, 0.59145..."
669,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,19,AutoGluon,"[0.684054496351183, 0.6333300270243154, 0.6506..."
670,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,33,AutoGluon,"[0.9192393552539837, 0.9430385201823036, 0.947..."
...,...,...,...,...
933,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,33,AutoGluon,"[0.9658837811297082, 0.9460477298688553, 0.947..."
934,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,28,AutoGluon,"[0.952194293948165, 0.9428353929941512, 0.9459..."
935,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,16,AutoGluon,"[0.989941075170802, 0.9893044640816396, 0.9833..."
936,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,8,AutoGluon,"[0.999847839824526, 0.9998279022141227, 0.9997..."


In [ ]:
# TODO List
# Bar Chart Normalized Error
# Critical Difference Diagram
# fANOVA (Optional)
# Plot AutoGluons Performance over Training Time (In Normalized Error, Rank, etc.)
# Bar Chart of Datasets Ordered by difficulty

In [30]:
import pandas as pd

df = pd.DataFrame({"group": ["A"]*4, "value": [10, 20, 15, 30]})
df["rank"] = df["value"].rank(ascending=False)

print(df)


  group  value  rank
0     A     10   4.0
1     A     20   2.0
2     A     15   3.0
3     A     30   1.0


In [7]:
df = pd.read_sql_query(f"SELECT *  FROM results", conn_ctr)
df

,hash_key,model,search_space_name,search_ids,experiment_id,search_space_hash,data_config_hash,data_id,best_configs,search_algo,timestamp,search_durations,prediction_time,train_samples,test_samples,mean_absolute_error,mean_squared_error,r2_score,mean_absolute_percentage_error,root_mean_squared_error
0,82b19201ed5d1f4ef5899ef75d85652904925714adbf59...,ElasticNet,ElasticNet,"['6c1ce6cc-91af-44b8-a855-0267a0af333c', 'f21a...",dc9e576b-d8e3-4d81-985d-05fcef289b1c,50450bd063d3a9ade1ac1ea55650ea4e9605bae8114751...,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,24,"[{'alpha': 0.1067707176877, 'fit_intercept': T...",HEBO,1.767121e+09,"[669.6938490867615, 164.94125318527222, 158.98...","[0.008745431900024414, 0.016772031784057617, 0...","[9622, 9622, 9623, 9623, 9623, 9623, 9623, 962...","[1070, 1070, 1069, 1069, 1069, 1069, 1069, 106...","[1968.4123310205591, 1751.1937394746506, 1805....","[98180102.7720658, 8040609.67308422, 10527923....","[0.10567121376356414, 0.7244679272319543, 0.49...","[0.3777477469714053, 0.3972547755465607, 0.398...","[9908.587324743412, 2835.5968812728333, 3244.6..."
1,b988af29cb7ea58547b68c3c51632d8ab46a1f47c04ad7...,ExtraTrees,ExtraTrees,"['876a537f-9603-4c58-bc07-fffd9643f54f', '49f9...",53a6f02e-8133-4e9b-a6b9-82a6ca5b6ee8,ea96e45d1a536c29c34a442f834d11737b4b9de233d0fc...,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,30,"[{'criterion': 'absolute_error', 'max_features...",HEBO,1.767122e+09,"[666.099426984787, 204.10563349723816, 220.173...","[0.011622428894042969, 0.011568069458007812, 0...","[465, 465, 465, 465, 465, 465, 465, 466, 466, ...","[52, 52, 52, 52, 52, 52, 52, 51, 51, 51]","[34.236463461538456, 7.6428442307692315, 20.27...","[23416.111975463748, 231.89751376894233, 10753...","[-0.024123602331189042, -0.10398036220475104, ...","[1.8246870857939444e+16, 5832326072046562.0, 8...","[153.023239984859, 15.228181564748377, 103.698..."
2,e0b96bbe11050760124451b70b22c33697df485b5f06f2...,Dummy,Dummy,"['754a8ac7-be5a-4ce0-a15d-94560ddc5de2', '5516...",deab1856-cd96-42f9-96fb-e5c136f7db1b,c177dd0e73312c7e92964fd0de7766f44fa2231a5594df...,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,9,"[{'strategy': 'mean'}, {'strategy': 'mean'}, {...",HEBO,1.767122e+09,"[663.1099207401276, 206.2958526611328, 209.773...","[2.7418136596679688e-05, 2.6941299438476562e-0...","[4408, 4408, 4408, 4408, 4408, 4408, 4408, 440...","[490, 490, 490, 490, 490, 490, 490, 490, 489, ...","[0.6711322641579317, 0.6976138005111299, 0.643...","[0.742628675406678, 0.8365257774637617, 0.7214...","[-0.0005395067877793647, -4.1781091721926344e-...","[0.11793304293457077, 0.12385474631041596, 0.1...","[0.8617590587900298, 0.9146178313720773, 0.849..."
3,7783f30279a1eedf2dcc8f3f5172b51c45f9cbc420d438...,ElasticNet,ElasticNet,"['d3572876-1273-4920-88b1-455126d7a3f1', 'c00f...",ccb52de2-e70e-4165-9f05-b96f61ab37f8,50450bd063d3a9ade1ac1ea55650ea4e9605bae8114751...,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,28,"[{'alpha': 0.0002355863843, 'fit_intercept': T...",HEBO,1.767122e+09,"[616.8554286956787, 194.33259296417236, 215.47...","[0.0007991790771484375, 0.0008118152618408203,...","[1108, 1108, 1109, 1109, 1109, 1109, 1109, 110...","[124, 124, 123, 123, 123, 123, 123, 123, 123, ...","[16.224560838078993, 17.522856765589555, 16.38...","[415.42193377163096, 472.56617115069656, 421.7...","[0.9496890647722223, 0.940504449782539, 0.9469...","[0.022749131685365534, 0.025371540162948136, 0...","[20.381902113679942, 21.73858714706861, 20.536..."
4,b3e13752c7f7515347f0880dd29aecf1d55219fa60ddd5...,BayesianRidge,BayesianRidge,"['01af5b71-58bb-4079-bfd5-4618f10f6572', 'ce20...",511da64a-7b0d-4e4f-bea8-631898309b73,e1cae70c1ad874346b049602b4585d58c088ed90205870...,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,24,"[{'alpha_1': 0.68244768024, 'alpha_2': 9.43314...",HEBO,1.767122e+09,"[670.1938514709473, 240.96088814735413, 204.88...","[0.0018811225891113281, 0.0015935897827148438,...","[9622, 9622, 9623, 9623, 9623, 9623, 9623, 962...","[1070